In [24]:
# print("Number of rows: {}".format(SUBRELOBJ.shape))
# print("Columns : {}".format(SUBRELOBJ.columns.tolist()))

### Implementing a quick lookup mechanism

In [29]:
import re
pattern = r'.*Printing Circuits.*'  # This matches "Corrosion Protection" followed by any characters
matches = [ent for ent in unique_entities_set if re.match(pattern, ent)]

if matches:
    print(f"Entities that match the pattern '{pattern}': {matches}")
else:
    print(f"No entities found that match the pattern '{pattern}'.")

No entities found that match the pattern '.*Printing Circuits.*'.


Let's see a couple of random samples

In [26]:
# SUBRELOBJ.sample(5)

## Querying CSV File

In [27]:
# def query_subrelobj(query, rel, k=20):
    
#     def get_relevant_entities(query):
#         entities = []
#         for ent in unique_entities:
#             if query.lower() in ent.lower():
#                 entities.append(ent)
#         return entities
    
#     def query_df(entities, rel):
#         filtered_df = SUBRELOBJ[(SUBRELOBJ['Subject'].isin(entities)) & (SUBRELOBJ['Rel'] == rel)]
#         grouped_df = filtered_df.groupby('Object')['Count'].sum().reset_index()
#         sorted_df = grouped_df.sort_values(by='Count', ascending=False)
#         objects_and_counts = sorted_df.set_index('Object')['Count']
#         num = min(k, len(objects_and_counts))
#         return objects_and_counts[:num]
    
#     relevant_entities = get_relevant_entities(query)
    
#     if len(relevant_entities)<1:
#         print("No relevant query results")
#         return None
    
#     results = query_df(relevant_entities, rel)
#     return results

# def query_sub(query, k=15):
    
#     def get_relevant_entities(query):
#         entities = []
#         for ent in unique_entities:
#             if query.lower() in ent.lower():
#                 entities.append(ent)
#         return entities
    
#     def query_df(entities):
#         filtered_df = SUBRELOBJ[(SUBRELOBJ['Subject'].isin(entities))]
# #         grouped_df = filtered_df.groupby('Object')['Count'].sum().reset_index()
# #         sorted_df = grouped_df.sort_values(by='Count', ascending=False)
# #         objects_and_counts = sorted_df.set_index('Object')['Count']
# #         num = min(k, len(objects_and_counts))
#         return filtered_df
    
#     relevant_entities = get_relevant_entities(query)
    
#     if len(relevant_entities)<1:
#         print("No relevant query results")
#         return None
    
#     results = query_df(relevant_entities)
#     return results

Finding all applications of a material 'X'

In [28]:
# results = query_subrelobj('Anti-Corrosion Coatings', 'APL-CHM')
# results

In [29]:
# sub_results = query_sub('Corrosion Protection')
# sub_results

### Finding materials (CHM) and properties (PRO) based on an extracted application (APL)

In [1]:
import sys
# Add the desired path to sys.path
sys.path.append('C:\\Users\\VENKATESH\\anaconda3\\envs\\py310\\Lib\\site-packages')
# sys.path.append('C:\Users\VENKATESH\anaconda3\envs\py310\Lib\site-packages')
print(sys.path)

['C:\\Users\\VENKATESH\\Desktop\\Materials_Discovery', 'C:\\Users\\VENKATESH\\anaconda3\\python39.zip', 'C:\\Users\\VENKATESH\\anaconda3\\DLLs', 'C:\\Users\\VENKATESH\\anaconda3\\lib', 'C:\\Users\\VENKATESH\\anaconda3', '', 'C:\\Users\\VENKATESH\\anaconda3\\lib\\site-packages', 'C:\\Users\\VENKATESH\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\VENKATESH\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\VENKATESH\\anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\VENKATESH\\anaconda3\\envs\\py310\\Lib\\site-packages']


In [2]:
import os
import dotenv
from dotenv import load_dotenv
import time
import openai
from openai import OpenAI
import random 
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import re
import json

# Load environment variables from .env file
load_dotenv(dotenv_path='C:/Users/VENKATESH/Desktop/Legal_Reasoning/mykey.env')
# load_dotenv(dotenv_path='C:/Users/VENKATESH/Desktop/Legal_Reasoning/mykey.env')

# Now you can access the environment variable
api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=api_key)

In [3]:
import requests
# OpenAI API call handling for timout issues
def get_text_completion_from_GPT(system_prompt, json_format=False, model="gpt-4o-mini", max_retries=3, retry_delay=2):
    for attempt in range(max_retries):
        try:
            messages = [{"role": "system", "content": system_prompt}]
            if json_format:
                response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    response_format = {"type": "json_object"},
                    temperature=0,  # Degree of randomness of the model's output
                    frequency_penalty=0,
                    presence_penalty=0,
                    seed = 42
                )
                return response.choices[0].message.content
            else:
                response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                    temperature=0,  # Degree of randomness of the model's output
                    frequency_penalty=0,
                    presence_penalty=0,
                    seed = 42
                )
                return response.choices[0].message.content
        except (requests.exceptions.Timeout, requests.exceptions.RequestException) as e:
            # print(f"Attempt {attempt + 1} failed. Retrying after {retry_delay} seconds.")
            time.sleep(retry_delay)
    # If all attempts fail, raise the last exception
    raise e

In [4]:
SUBRELOBJ = pd.read_csv('SUBRELOBJ.csv')
unique_entities = list(set(SUBRELOBJ['Subject'].tolist() + SUBRELOBJ['Object'].tolist()))
unique_entities_set = set(SUBRELOBJ['Subject'].tolist() + SUBRELOBJ['Object'].tolist())
print("Number of unique entities : {}".format(len(unique_entities)))
print("Number of unique entities (set) : {}".format(len(unique_entities_set)))

Number of unique entities : 70002
Number of unique entities (set) : 70002


In [5]:
# Filter rows where 'Rel' is either 'APL-CHM', 'CHM-APL', 'APL-PRO', or 'PRO-APL'
APL_CHM_PRO_SUBRELOBJ = SUBRELOBJ[(SUBRELOBJ['Rel'] == 'APL-CHM') | (SUBRELOBJ['Rel'] == 'CHM-APL') | 
                 (SUBRELOBJ['Rel'] == 'APL-PRO') | (SUBRELOBJ['Rel'] == 'PRO-APL')]

In [ ]:
# # TODO: Write the logic to extract the goal_statement automatically fromt the csv file
# df = pd.read_csv('KG_MatDis.csv')
# # output_file = 'C:/Users/VENKATESH/Desktop/Materials_Discovery/KG_MatDis_Output.csv'
# # output_df = pd.read_csv(output_file)
# KG_augmented_output_list = []
# for index, row in df.iterrows():
#     goal_statement = row['Goal Statement']
#     application_extraction_system_prompt = f"""You are an expert Material Scientist. Your task is to extract the 'applications' embedded in the goal statement provided to you. Extract the applications and print them separated by commas. The initial letter of every word in the applications list should be a capital letter.
#     The extracted applications should be within two or three words. 
#     Provided Goal Statement:
#     {goal_statement}
#     """
#     extracted_application_list = get_text_completion_from_GPT(application_extraction_system_prompt)
#     # 'apl_list' is the list of extracted APLs from the goal statement
#     apl_list = extracted_application_list.split(', ')
#     # apl_list = ['Corrosion Protection', 'Metallic Structures', 'Offshore Environments', 'Mechanical Damage', 'Environmental Factors', 'Self-Healing Agents', 'Large-Scale Application', 'Spray Painting', 'Corrosion Resistance', 'Multiple Healing Cycles']  # Example list of extracted APLs

#     # Initialize an empty dictionary to store the results
#     result_dict = {}

#     final_mat_prop_extracted_list = ""

#     # Iterate over each application in the apl_list
#     for apl in apl_list:
#         # Define a regex pattern to check if the application exists as a unique entity in the dataframe
#         pattern = r'^' + re.escape(apl) + r'.*'  # Escape special characters and match the application as a prefix

#         # Check if the application is present as a unique entity in the dataframe
#         matches = [ent for ent in unique_entities_set if re.match(pattern, ent)]

#         # Only proceed if the application exists in the unique entities
#         if matches:
#     #         print(f"\nEntities that match the pattern '{apl}': {matches}")

#             # Check if the application is present in the 'Subject' or 'Object' column as an 'APL' entity in the dataframe
#             is_apl_in_df = False

#             # Check if the APL is present in any 'APL' relationships (e.g., APL-CHM, APL-PRO, etc.)
#             for index, row in APL_CHM_PRO_SUBRELOBJ.iterrows():
#                 # Check for APL in 'Subject' when the relationship starts with APL
#                 if apl == row['Subject'] and row['Rel'].startswith('APL'):
#                     is_apl_in_df = True
#                     break
#                 # Check for APL in 'Object' when the relationship ends with APL
#                 elif apl == row['Object'] and row['Rel'].endswith('APL'):
#                     is_apl_in_df = True
#                     break

#             # If the application is present as an APL entity, proceed with CHM and PRO search
#             if is_apl_in_df:
#                 # Initialize lists to store the matched materials and properties
#                 matched_materials = []
#                 matched_properties = []

#                 # Loop over the dataframe to find materials (CHM) and properties (PRO) based on extracted APLs
#                 for index, row in APL_CHM_PRO_SUBRELOBJ.iterrows():
#                     # For 'APL-CHM' pairs, extract the material from 'Object' if APL is in 'Subject'
#                     if row['Rel'] == 'APL-CHM' and row['Subject'] == apl:
#                         matched_materials.append(row['Object'])

#                     # For 'CHM-APL' pairs, extract the material from 'Subject' if APL is in 'Object'
#                     elif row['Rel'] == 'CHM-APL' and row['Object'] == apl:
#                         matched_materials.append(row['Subject'])

#                     # For 'APL-PRO' pairs, extract the property from 'Object' if APL is in 'Subject'
#                     elif row['Rel'] == 'APL-PRO' and row['Subject'] == apl:
#                         matched_properties.append(row['Object'])

#                     # For 'PRO-APL' pairs, extract the property from 'Subject' if APL is in 'Object'
#                     elif row['Rel'] == 'PRO-APL' and row['Object'] == apl:
#                         matched_properties.append(row['Subject'])

#                 # Convert the matched materials and properties to unique lists (to remove duplicates)
#                 matched_materials = list(set(matched_materials))
#                 matched_properties = list(set(matched_properties))

#                 # Add the results to the dictionary under the application key
#                 result_dict[apl] = {
#                     'materials': matched_materials,
#                     'properties': matched_properties
#                 }
#                 final_mat_prop_extracted_list  = final_mat_prop_extracted_list  + f"\nTo satisfy the application of {apl}, the potential materials which can be explored are: {result_dict[apl]['materials']} \nTo satisfy the application of {apl}, The properties which can be explored are: {result_dict[apl]['properties']} "

#             else:
#     #             print(f"\nApplication '{apl}' is not found as an 'APL' entity in the dataframe. No further search conducted.")
#                 pass

#         else:
#             # If the application is not found as a unique entity, skip further searches
#             pass

#     # You can pass 'final_mat_prop_extracted_list' directly to an LLM or any other processing system.
# #     print(final_mat_prop_extracted_list)
#     materials_and_properties_extraction_system_prompt = f"""You are an expert Material Scientist. 
#     Your task is to go through a list of materials and properties extracted based on a particular application from a Knowledge Graph and then extract the most commonly occurring materials and properties which satisfy these multiple applications. 
#     You have been provided with a goal statements from which appropriate target applications, which are to be explored, have been already extracted. 

#     You must follow the instructions below:
#     ### Instructions:
#     1. Extract the commonly occuring materials and properties across multiple applications.
#     2. If there is only application in the extracted applications and their corresponding materials and properties, cite the materials and properties which will most ideally satisfy the goal statement.
#     3. If there are no commonalities between the materials and properties of extracted applications, choose the most appropriate materials and properties from multiple applications which ideally satisfy the goal statement.
#     4. If there is no data extracted from the Knowledge Graph, then return an empty json dictionary. DO NOT ADD any materials or properties from your parametric knowledge.
    
#     ### Provided Goal Statement:
#     {goal_statement}

#     ### Extracted Applications from Knowledge Graph corresponding materials and properties:
#     {final_mat_prop_extracted_list }

#     Provide your response in a strict json format with the following two keys:
#     KG Suggested Materials: <List of extracted materials and the explanation of the reasoning behind choosing them as key-value pairs>
#     KG Suggested Properties: <List of extracted properties and the explanation of the reasoning behind choosing them as key-value pairs>
#     """
#     json_format = True # To enforce JSON format
#     KG_extracted_materials_properties = get_text_completion_from_GPT(materials_and_properties_extraction_system_prompt, json_format)
#     KG_augmented_output_list.append(KG_extracted_materials_properties)
#     print(f"Goal: {goal_statement}:\n\n{KG_extracted_materials_properties}")
# df['KG_augmented_Output'] = KG_augmented_output_list   
# df.to_csv('KG_MatDis.csv', index=False)


# TODO: Write the logic to extract the goal_statement automatically fromt the csv file
df = pd.read_csv('KG_MatDis.csv')
# output_file = 'C:/Users/VENKATESH/Desktop/Materials_Discovery/KG_MatDis_Output.csv'
# output_df = pd.read_csv(output_file)
KG_augmented_output_list = []
for index, row in df.iterrows():
    goal_statement = row['Goal Statement']
    application_extraction_system_prompt = f"""You are an expert Material Scientist. Your task is to extract the 'applications' embedded in the goal statement provided to you. Extract the applications and print them separated by commas. The initial letter of every word in the applications list should be a capital letter.
    The extracted applications should be within two or three words. 
    Provided Goal Statement:
    {goal_statement}
    """
    extracted_application_list = get_text_completion_from_GPT(application_extraction_system_prompt)
    # 'apl_list' is the list of extracted APLs from the goal statement
    apl_list = extracted_application_list.split(', ')
    print(f"The applications extracted by the LLM are: {apl_list}\n\n")
    # apl_list = ['Corrosion Protection', 'Metallic Structures', 'Offshore Environments', 'Mechanical Damage', 'Environmental Factors', 'Self-Healing Agents', 'Large-Scale Application', 'Spray Painting', 'Corrosion Resistance', 'Multiple Healing Cycles']  # Example list of extracted APLs

    # Initialize an empty dictionary to store the results
    result_dict = {}

    final_mat_prop_extracted_list = ""

    # Iterate over each application in the apl_list
    for apl in apl_list:
        # Define a regex pattern to check if the application exists as a unique entity in the dataframe
        pattern = r'^' + re.escape(apl) + r'.*'  # Escape special characters and match the application as a prefix

        # Check if the application is present as a unique entity in the dataframe
        matches = [ent for ent in unique_entities_set if re.match(pattern, ent)]

        # Only proceed if the application exists in the unique entities
        if matches:
    #         print(f"\nEntities that match the pattern '{apl}': {matches}")

            # Check if the application is present in the 'Subject' or 'Object' column as an 'APL' entity in the dataframe
            is_apl_in_df = False

            # Check if the APL is present in any 'APL' relationships (e.g., APL-CHM, APL-PRO, etc.)
            for index, row in APL_CHM_PRO_SUBRELOBJ.iterrows():
                # Check for APL in 'Subject' when the relationship starts with APL
                if apl == row['Subject'] and row['Rel'].startswith('APL'):
                    is_apl_in_df = True
                    break
                # Check for APL in 'Object' when the relationship ends with APL
                elif apl == row['Object'] and row['Rel'].endswith('APL'):
                    is_apl_in_df = True
                    break

            # If the application is present as an APL entity, proceed with CHM and PRO search
            if is_apl_in_df:
                # Initialize lists to store the matched materials and properties
                matched_materials = []
                matched_properties = []

                # Loop over the dataframe to find materials (CHM) and properties (PRO) based on extracted APLs
                for index, row in APL_CHM_PRO_SUBRELOBJ.iterrows():
                    # For 'APL-CHM' pairs, extract the material from 'Object' if APL is in 'Subject'
                    if row['Rel'] == 'APL-CHM' and row['Subject'] == apl:
                        matched_materials.append(row['Object'])

                    # For 'CHM-APL' pairs, extract the material from 'Subject' if APL is in 'Object'
                    elif row['Rel'] == 'CHM-APL' and row['Object'] == apl:
                        matched_materials.append(row['Subject'])

                    # For 'APL-PRO' pairs, extract the property from 'Object' if APL is in 'Subject'
                    elif row['Rel'] == 'APL-PRO' and row['Subject'] == apl:
                        matched_properties.append(row['Object'])

                    # For 'PRO-APL' pairs, extract the property from 'Subject' if APL is in 'Object'
                    elif row['Rel'] == 'PRO-APL' and row['Object'] == apl:
                        matched_properties.append(row['Subject'])

                # Convert the matched materials and properties to unique lists (to remove duplicates)
                matched_materials = list(set(matched_materials))
                matched_properties = list(set(matched_properties))

                # Add the results to the dictionary under the application key
                result_dict[apl] = {
                    'materials': matched_materials,
                    'properties': matched_properties
                }
                final_mat_prop_extracted_list  = final_mat_prop_extracted_list  + f"\nTo satisfy the application of {apl}, the potential materials which can be explored are: {result_dict[apl]['materials']} \nTo satisfy the application of {apl}, The properties which can be explored are: {result_dict[apl]['properties']} "

            else:
    #             print(f"\nApplication '{apl}' is not found as an 'APL' entity in the dataframe. No further search conducted.")
                pass

        else:
            # If the application is not found as a unique entity, skip further searches
            pass

    # You can pass 'final_mat_prop_extracted_list' directly to an LLM or any other processing system.
    print(f"The materials and properties extracted are:\n{final_mat_prop_extracted_list}\n\n")
    materials_and_properties_extraction_system_prompt = f"""You are an expert Material Scientist. 
    Your task is to go through a list of materials and properties extracted based on a particular application from a Knowledge Graph and then add explanations of reasoning behind these 20 extracted materials and properties. 
    You have been provided with a goal statements from which appropriate target applications, which are to be explored, have been already extracted. 

    You must follow the instruction below:
    ### Instruction:
    1. Extract only the top twenty materials and properties if there are more twenty of them.
    2. If there is no data extracted from the Knowledge Graph for an extracted application, then return an empty json dictionary. DO NOT ADD any materials or properties from your parametric knowledge.
    3. DO NOT create any 'application' terms from your own parametric knowledge. 
    
    ### Provided Goal Statement:
    {goal_statement}

    ### Extracted Applications from Knowledge Graph corresponding materials and properties:
    {final_mat_prop_extracted_list}

    Provide your response in a strict json format with the following format:
    {{
    "<Name of the extracted application>": 
        {{
            KG Suggested Materials: <List of extracted materials and the explanation of the reasoning behind choosing them as key-value pairs>,
            KG Suggested Properties: <List of extracted properties and the explanation of the reasoning behind choosing them as key-value pairs>
        }}
    }}
    """
    json_format = True # To enforce JSON format
    KG_extracted_materials_properties = get_text_completion_from_GPT(materials_and_properties_extraction_system_prompt, json_format)
    KG_augmented_output_list.append(KG_extracted_materials_properties)
    print(f"Goal: {goal_statement}:\n\n{KG_extracted_materials_properties}\n")
df['KG_augmented_Full_Output'] = KG_augmented_output_list   
df.to_csv('KG_MatDis.csv', index=False)